In [11]:
import mido
import pandas as pd


# Function to convert MIDI note number to frequency
def midi_note_to_frequency(note_number):
    # MIDI note number to frequency conversion formula
    return 440.0 * (2.0 ** ((note_number - 69) / 12.0))


# a dictionary to store midi-conversion to key index
midi_note_to_key_index_dict = {
    0: 0,
    1: 20,
    2: 1,
    3: 21,
    4: 2,
    5: 3,
    6: 23,
    7: 4,
    8: 24,
    9: 5,
    10: 25,
    11: 6,
    12: 10,
    13: 30,
    14: 11,
    15: 31,
    16: 12,
    17: 13,
    18: 33,
    19: 14,
    20: 34,
    21: 15,
    22: 35,
    23: 16,
    24: 17,
    25: 37,
    26: 18,
    27: 38,
    28: 19,
}


def midi_note_to_key_index(note_number):
    # MIDI note number to piano display coordinates conversion formula
    #
    # Because the piano display starts at C3 and goes through E5, which
    # is not an integer multiple of an octave above C3, the conversion
    # only maps to the 12 keys from C3 (played with z) to B3 (played with m)
    # and the 12 keys from C4 (played with q) to B4 (played with u)
    #
    # The conversion is done with modulo arithmatic
    #
    # The entire midi note range, from 0 to 127, is mapped in two octave
    # chunks to 24 keys.
    note_number = (note_number - 60) % 24
    return midi_note_to_key_index_dict[note_number]


# values from the 01_SineEnv_Piano.cpp file from the allolib_playground tutorials folder
keyWidth = 116
keyHeight = 296
keyPadding = 2


# translated from the 01_SineEnv_Piano.cpp file from the allolib_playground tutorials folder
def key_index_to_coordinates(key_index):
    is_black_key = False
    if key_index >= 20:
        key_index -= 20
        is_black_key = True

    w = keyWidth
    h = keyHeight
    x = (keyWidth + keyPadding * 2) * (key_index % 10) + keyPadding
    y = 0

    if is_black_key == True:
        x += keyWidth * 0.5
        y += keyHeight * 0.5
        h *= 0.5

    if key_index >= 10:
        y += keyHeight + keyPadding * 2

    return (w, h, x, y)


# Load MIDI file
midi_file = mido.MidiFile("phineasandferb.mid")

# Create a list to hold note information
notes_list = []

# Initialize a variable to keep track of the current time
current_time = 0

# Dictionary to keep track of note start times
note_start_times = {}

# Dictionary to keep track of note key coordinates
note_key_coordinates = {}

# offset for program start delay
offset = 0.1

# Parameters for the SineEnv Synthesizer
amplitude = 0.2
attack_time = 0.034
release_time = 0.458
pan = 0

# Iterate over all messages in the MIDI file
for msg in midi_file:
    # Update the current time
    current_time += msg.time
    if msg.type == "note_on" and msg.velocity > 0:
        # Note on message, start of a note
        note_start_times[msg.note] = current_time
        # Store the key coordinates for the note
        note_key_coordinates[msg.note] = key_index_to_coordinates(
            midi_note_to_key_index(msg.note)
        )
    elif msg.type == "note_off" or (msg.type == "note_on" and msg.velocity == 0):
        # Note off message, end of a note
        if msg.note in note_start_times:
            note_start_time = note_start_times.pop(msg.note)
            note_end_time = current_time
            frequency = midi_note_to_frequency(msg.note)
            key_coordinates = note_key_coordinates.pop(msg.note)
            # print(msg.note, ' ', key_coordinates)
            notes_list.append(
                [
                    "@",
                    note_start_time + 0.1,
                    note_end_time + 0.1,
                    "DemoVoice",
                    amplitude,
                    frequency,
                    attack_time,
                    release_time,
                    pan,
                ]
            )

# Create a DataFrame with the notes list
df = pd.DataFrame(
    notes_list,
    columns=[
        " ",
        "start_time",
        "end_time",
        "DemoVoice",
        "amplitude",
        "frequency",
        "attack_time",
        "release_time",
        "pan",
    ],
)

# Display the DataFrame
# print(df)

In [12]:
df

,,start_time,end_time,DemoVoice,amplitude,frequency,attack_time,release_time,pan
0,@,0.100000,0.183333,DemoVoice,0.2,207.652349,0.034,0.458,0
1,@,0.266667,0.350000,DemoVoice,0.2,207.652349,0.034,0.458,0
2,@,0.433333,0.516666,DemoVoice,0.2,349.228231,0.034,0.458,0
3,@,0.600000,0.683333,DemoVoice,0.2,349.228231,0.034,0.458,0
4,@,0.433333,0.683333,DemoVoice,0.2,138.591315,0.034,0.458,0
...,...,...,...,...,...,...,...,...,...
461,@,52.849947,52.933281,DemoVoice,0.2,880.000000,0.034,0.458,0
462,@,53.016614,53.099947,DemoVoice,0.2,880.000000,0.034,0.458,0
463,@,53.849946,53.933280,DemoVoice,0.2,207.652349,0.034,0.458,0
464,@,53.849946,53.933280,DemoVoice,0.2,415.304698,0.034,0.458,0


In [13]:
# replace end_time with duration without changing the order of the columns
duration = df["end_time"] - df["start_time"]
df = df.drop(columns=["end_time"])
df.insert(2, "duration", duration)

df

,,start_time,duration,DemoVoice,amplitude,frequency,attack_time,release_time,pan
0,@,0.100000,0.083333,DemoVoice,0.2,207.652349,0.034,0.458,0
1,@,0.266667,0.083333,DemoVoice,0.2,207.652349,0.034,0.458,0
2,@,0.433333,0.083333,DemoVoice,0.2,349.228231,0.034,0.458,0
3,@,0.600000,0.083333,DemoVoice,0.2,349.228231,0.034,0.458,0
4,@,0.433333,0.250000,DemoVoice,0.2,138.591315,0.034,0.458,0
...,...,...,...,...,...,...,...,...,...
461,@,52.849947,0.083333,DemoVoice,0.2,880.000000,0.034,0.458,0
462,@,53.016614,0.083333,DemoVoice,0.2,880.000000,0.034,0.458,0
463,@,53.849946,0.083333,DemoVoice,0.2,207.652349,0.034,0.458,0
464,@,53.849946,0.083333,DemoVoice,0.2,415.304698,0.034,0.458,0


In [14]:
# convert dataframe to a space separated file
df.to_csv(
    "bin/DemoVoice-data/phineasandferb.synthSequence",
    sep=" ",
    index=False,
    header=False,
)